In [0]:
import pandas as pd

In [0]:
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [0]:
dbutils.fs.unmount("/mnt/alv")

/mnt/alv has been unmounted.
Out[3]: True

In [0]:
def mount_blob(storage_account_name, access_key):
  #access_key = dbutils.secrets.get(scope="storage", key=storage_account_name)
  mount_list = [
    {
      "folder_name": "alv",
      "message" : "Mount Azure Blob Storage (alv) in DBFS..."
    }       
  ]
  
  for item in mount_list:
    print(item['message'])    
    folder_name = item['folder_name']
    mount_point = '/mnt/{}'.format(folder_name)
    blob_path = 'wasbs://{}@{}.blob.core.windows.net'.format(folder_name, storage_account_name)
    config = "fs.azure.account.key.{}.blob.core.windows.net".format(storage_account_name)
    try:
      dbutils.fs.ls(mount_point)
      print('{} has been mounted.'.format(mount_point))
    except:
      print('DBFS Mount does not exist, creating mount {}...'.format(mount_point))
      
      dbutils.fs.mount(
        source = blob_path,
        mount_point = mount_point,
        extra_configs = {
          config:access_key
        }
      )

In [0]:
storageAccountName = 'bidseadlg2'
# blobContainerName = 'alv'
storageAccountAccessKey = 'XXXXXX'
mount_blob(storageAccountName, storageAccountAccessKey)

Mount Azure Blob Storage (alv) in DBFS...
DBFS Mount does not exist, creating mount /mnt/alv...

In [0]:
def read_data_to_dataframe(file_location):
  file_type = "csv"
  # CSV options
  infer_schema = "false"
  first_row_is_header = "true"
  delimiter = ","
  encoding = 'utf-8'

  # The applied options are for CSV files. For other file types, these will be ignored.
  df = spark.read.format(file_type) \
    .option("inferSchema", infer_schema) \
    .option("header", first_row_is_header) \
    .option("sep", delimiter) \
    .option("encoding", encoding) \
    .load(file_location)
  return df

In [0]:
def output_csv_file(df, path):
   (df.coalesce(1) # uncomment this line to ensure that the data is written to just one file
   .write
   .option("header", "true")
   .option("sep", ",")
   .option("encoding", "utf-8")
   .option("ESCAPE quote", '"')
   .mode("overwrite") # overwrite any pre-existing data at the path that we're writing to
   .csv(path)
  )

In [0]:
file_list = []
df_list = []
folder_path = '/mnt/alv/'
files = dbutils.fs.ls(folder_path + 'TEST')

for f in files:
  file_list.append(f.name)
  print(file_list)
  df = read_data_to_dataframe(f.path).toPandas()
  df = df.rename(columns={df.columns[len(df.columns)-1]: 'SourceFile'})
  df_list.append(df)

print(df.head())

['TEST_CN20 _201901.csv']
['TEST_CN20 _201901.csv', 'TEST_CN20_201902.csv']
 工廠 申請單號 項次 報廢狀態 ... 除帳物料文件 除帳數量 除帳內文 SourceFile
0 CN20 A180922021 1 除帳 ... 4901310555 950 版本升級 TEST_CN20_201902.csv
1 CN20 A180922021 2 除帳 ... 4901310555 1000 版本升級 TEST_CN20_201902.csv
2 CN20 A180922021 3 除帳 ... 4901310555 5670 版本升級 TEST_CN20_201902.csv
3 CN20 A180922021 4 除帳 ... 4901310555 150 版本升級 TEST_CN20_201902.csv
4 CN20 A180922021 5 除帳 ... 4901310555 300 版本升級 TEST_CN20_201902.csv

[5 rows x 36 columns]

In [0]:
df_combine = pd.concat(df_list)
df_combine = df_combine.dropna(axis = 1, how = 'all')
df_combine.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18 entries, 0 to 8
Data columns (total 31 columns):
 # Column Non-Null Count Dtype 
--- ------ -------------- ----- 
 0 工廠 18 non-null object
 1 申請單號 18 non-null object
 2 項次 18 non-null object
 3 報廢狀態 18 non-null object
 4 申請人員 18 non-null object
 5 成本中心 18 non-null object
 6 客戶代號 18 non-null object
 7 客戶名稱 18 non-null object
 8 物料 18 non-null object
 9 物料說明 18 non-null object
 10 批號 18 non-null object
 11 移倉物料文件 18 non-null object
 12 數量 18 non-null object
 13 SKU 18 non-null object
 14 每單位量 18 non-null object
 15 報廢原因 18 non-null object
 16 原因說明 18 non-null object
 17 幣別22 18 non-null object
 18 報廢金額(成本) 18 non-null object
 19 CNY報廢金額(成本) 18 non-null object
 20 結算金額 18 non-null object
 21 CNY結算金額 18 non-null object
 22 幣別27 18 non-null object
 23 報廢金額(銷售) 18 non-null object
 24 幣別29 18 non-null object
 25 本國貨幣金額 18 non-null object
 26 過帳日期 18 non-null object
 27 除帳物料文件 18 non-null object
 28 除帳數量 18 non-null object
 29 除帳內文 18 non-null object
 30 SourceFile 18 non-null object
dtypes: object(31)
memory usage: 4.5+ KB

In [0]:
# pandas dataframe to spark dataframe
df_spark = spark.createDataFrame(df_combine)
# display(df_spark)

In [0]:
write_path = "{}/TEST_merge".format(folder_path)
output_csv_file(df_spark, write_path)

tmp_files = dbutils.fs.ls(write_path)
output_file = [x for x in tmp_files if x.name.startswith("part-")]

# Move the wrangled-data CSV file from a sub-folder (wrangled_data_folder) to the root of the blob container
# While simultaneously changing the file name
dbutils.fs.mv(output_file[0].path, "{}/TEST.csv".format(folder_path))
dbutils.fs.rm(write_path, recurse = True)

Out[11]: True